# Brain Decoding

## Importing  basic libraries

In [1]:
import json
import os
import numpy as np
import nibabel as nib
from nibabel.testing import data_path
import pandas as pd
from ridge import bootstrap_ridge
from evalute import *
import scipy.stats as stats
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from Feature_extraction import *

import warnings
warnings.filterwarnings("ignore")

## Loading Fmri Data

In [2]:
ses2clip_train = {'007': ['01','02','03'],
            '008': ['05','06','07','08','09']
           }

ses2clip_test = {'009': ['04','10','11','12']}

fmri_original_train = []
fmri_original_test = []

In [3]:
for ses in list(ses2clip_train.keys()):
    for clip in ses2clip_train[ses]:
        fmri_filepath = '../../../Desktop/movie10.fmriprep/sub-01/ses-'+ses+'/func/sub-01_ses-'+ses+'_task-figures'+clip+'_run-1_space-fsLR_den-91k_bold.dtseries.nii'
        img = nib.load(fmri_filepath)
        fmri_original_train.append(img.get_data())
        
for ses in list(ses2clip_test.keys()):
    for clip in ses2clip_test[ses]:
        fmri_filepath = '../../../Desktop/movie10.fmriprep/sub-01/ses-'+ses+'/func/sub-01_ses-'+ses+'_task-figures'+clip+'_run-1_space-fsLR_den-91k_bold.dtseries.nii'
        img = nib.load(fmri_filepath)
        fmri_original_test.append(img.get_data())
        

## Loading Stimuli and extracting Features

In [4]:
video_feats_train = []
audio_feats_train = []

video_feats_test = []
audio_feats_test = []

## Extracting audio and video features from stimuli

# for ses in list(ses2clip_train.keys()):
#     for clip in ses2clip_train[ses]:
#         clip_filepath = '../../../Desktop/movie10.stimuli/figures/figures'+clip+'.mkv'
#         tr_time = json.load(open('../../../Desktop/movie10.fmriprep/sourcedata/movie10/task-figures'+clip+'_bold.json'))['RepetitionTime']
#         clip_video_feats, clip_audio_feats = extract_stimuli_features(clip_filepath,tr_time)
        
#         video_feats_train.append(clip_video_feats)
#         audio_feats_train.append(clip_audio_feats)
        
# for ses in list(ses2clip_test.keys()):
#     for clip in ses2clip_test[ses]:
#         clip_filepath = '../../../Desktop/movie10.stimuli/figures/figures'+clip+'.mkv'
#         tr_time = json.load(open('../../../Desktop/movie10.fmriprep/sourcedata/movie10/task-figures'+clip+'_bold.json'))['RepetitionTime']
#         clip_video_feats, clip_audio_feats = extract_stimuli_features(clip_filepath,tr_time)
        
#         video_feats_test.append(clip_video_feats)
#         audio_feats_test.append(clip_audio_feats)



## Loading pre-extracted features
for ses in list(ses2clip_train.keys()):
    for clip in ses2clip_train[ses]:
        video_feats_filepath = '../data/figures'+clip+'_video.npy'
        audio_feats_filepath = '../data/figures'+clip+'_audio.npy'
                
        video_feats_train.append(np.load(video_feats_filepath))
        audio_feats_train.append(np.load(audio_feats_filepath))
        
for ses in list(ses2clip_test.keys()):
    for clip in ses2clip_test[ses]:
        video_feats_filepath = '../data/figures'+clip+'_video.npy'
        audio_feats_filepath = '../data/figures'+clip+'_audio.npy'
                
        video_feats_test.append(np.load(video_feats_filepath))
        audio_feats_test.append(np.load(audio_feats_filepath))

## Creating Input and Target Matrices

In [5]:
k = 6

final_audio_feats_train = []
final_video_feats_train = []
final_fmri_train = []

final_audio_feats_test = []
final_video_feats_test = []
final_fmri_test = []

for i in range(len(audio_feats_train)):
    for j in range(k,audio_feats_train[i].shape[0]):
        final_audio_feats_train.append(np.concatenate(audio_feats_train[i][j-k:j]))
        final_video_feats_train.append(np.concatenate(video_feats_train[i][j-k:j]))
        final_fmri_train.append(fmri_original_train[i][j])
        
for i in range(len(audio_feats_test)):
    for j in range(k,audio_feats_test[i].shape[0]):
        final_audio_feats_test.append(np.concatenate(audio_feats_test[i][j-k:j]))
        final_video_feats_test.append(np.concatenate(video_feats_test[i][j-k:j]))
        final_fmri_testv.append(fmri_original_test[i][j])
    
final_fmri_train = stats.zscore(np.array(final_fmri_train))
final_audio_feats_train = stats.zscore(np.array(final_audio_feats_train))
final_video_feats_train = stats.zscore(np.array(final_video_feats_train))
final_fmri_test = stats.zscore(np.array(final_fmri_test))
final_audio_feats_test = stats.zscore(np.array(final_audio_feats_test))
final_video_feats_test = stats.zscore(np.array(final_video_feats_test))

In [6]:
final_fmri_train = np.nan_to_num(final_fmri_train)
final_fmri_test = np.nan_to_num(final_fmri_test)

## Training the Model

### Ridge Regression

In [22]:
alphas = np.logspace(1, 3, 10) # Equally log-spaced alphas between 10 and 1000. The third number is the number of alphas to test.
nboots = 1 # Number of cross-validation runs.
chunklen = 40 # 
nchunks = 20

X_train = final_fmri_train
X_test = final_fmri_test

y_train = final_audio_feats_train
y_test = final_audio_feats_test

wt, corr, alphas, bscorrs, valinds = bootstrap_ridge(X_train, y_train, X_test, y_test,
                                                     alphas, nboots, chunklen, nchunks,
                                                     singcutoff=1e-10, single_alpha=True)

In [24]:
y_pred = np.matmul(X_test,wt)

In [25]:
actual = y_test
predicted = y_pred

In [26]:
predicted = stats.zscore(predicted)
predicted = np.nan_to_num(predicted)

## Evaluation of Prediction

In [28]:
p_acc = acc_pairwise(actual, predicted)
r_acc = acc_rankBased(actual, predicted)

In [29]:
print(p_acc, r_acc)

0.980379746835443 0.9311708860759493
